In [1]:
import pandas as pd
df = pd.read_csv('stock.csv')
df.tail()

,Local time,open,high,low,close,volume
4323,14.10.2019 00:00:00.000 GMT+0300,108.372,108.525,108.033,108.404,122267.0400
4324,15.10.2019 00:00:00.000 GMT+0300,108.408,108.901,108.157,108.868,161800.8200
4325,16.10.2019 00:00:00.000 GMT+0300,108.878,108.878,108.565,108.792,178244.1800
4326,17.10.2019 00:00:00.000 GMT+0300,108.793,108.941,108.454,108.692,234278.7681
4327,18.10.2019 00:00:00.000 GMT+0300,108.706,108.724,108.388,108.433,148743.3600


In [2]:
indexZeros = df[df['volume'] == 0 ].index

df.drop(indexZeros, inplace=True)
df.loc[df['volume'] == 0]
df.isna().sum()

Local time    0
open          0
high          0
low           0
close         0
volume        0
dtype: int64

In [3]:
import numpy as np
import pandas_ta as ta

df['ATR'] = df.ta.atr(length=10)
df['RSI'] = df.ta.rsi()
df.dropna()


,Local time,open,high,low,close,volume,ATR,RSI
14,23.05.2003 00:00:00.000 GMT+0300,117.223,117.433,116.740,116.871,564114.1005,1.120226,43.218550
15,26.05.2003 00:00:00.000 GMT+0300,116.813,117.387,116.669,116.841,565039.0928,1.069574,42.935962
16,27.05.2003 00:00:00.000 GMT+0300,116.836,117.317,116.180,117.279,574499.0030,1.077851,48.255635
17,28.05.2003 00:00:00.000 GMT+0300,117.283,118.900,116.933,118.689,566670.9004,1.184562,60.894101
18,29.05.2003 00:00:00.000 GMT+0300,118.680,119.144,117.903,118.272,564631.6944,1.191203,56.498955
...,...,...,...,...,...,...,...,...
4323,14.10.2019 00:00:00.000 GMT+0300,108.372,108.525,108.033,108.404,122267.0400,0.683948,61.010164
4324,15.10.2019 00:00:00.000 GMT+0300,108.408,108.901,108.157,108.868,161800.8200,0.689953,65.223022
4325,16.10.2019 00:00:00.000 GMT+0300,108.878,108.878,108.565,108.792,178244.1800,0.652258,64.003170
4326,17.10.2019 00:00:00.000 GMT+0300,108.793,108.941,108.454,108.692,234278.7681,0.635732,62.350755


In [5]:
def Revsignal1(df1):
    df.dropna()
    df.reset_index(drop=True, inplace=True)
    
    length = len(df1)
    high = list(df1['high'])
    low = list(df1['low'])
    close = list(df1['close'])
    open = list(df1['open'])
    
    signal = [0] * length
    highdiff = [0] * length
    lowdiff = [0] * length
    bodydiff = [0] * length
    ratio1 = [0] * length
    ratio2 = [0] * length
    
    for row in range(length):
        
        highdiff[row] = high[row]-max(open[row], close[row])
        bodydiff[row] = abs(open[row]-close[row])
        if bodydiff[row]<0.002:
            bodydiff[row]=0.002
        lowdiff[row] = min(open[row],close[row])-low[row]
        ratio1[row] = highdiff[row]/bodydiff[row]
        ratio2[row] = lowdiff[row]/bodydiff[row]
        
        
        if (ratio1[row]>2.5 and lowdiff[row]<0.3*highdiff[row] and bodydiff[row]>0.03 and df.RSI[row]>50 and df.RSI[row]<70 ):
            signal[row] = 1
            
        elif (ratio2[row]>2.5 and highdiff[row]<0.23*lowdiff[row] and bodydiff[row]>0.03 and df.RSI[row]<55 and df.RSI[row]>30):
            signal[row]=2
    return signal
df['signal1'] = Revsignal1(df)
df[df['signal1']==2].count()
            
            
        

Local time    89
open          89
high          89
low           89
close         89
volume        89
ATR           89
RSI           89
signal1       89
dtype: int64

In [6]:
def mytarget(barsupfront, df1):

    length = len(df1)
    high = list(df1['high'])
    low = list(df1['low'])
    close = list(df1['close'])
    open = list(df1['open'])
    datr = list(df1['ATR'])
    trendcat = [0] * length
    

    for line in range (0,length-barsupfront-1):
        valueOpenLow = 0
        valueOpenHigh = 0
        
        highdiff = high[line]-max(open[line],close[line])
        bodydiff = abs(open[line]-close[line])
        
        pipdiff = datr[line]*1. #highdiff*1.3 #for SL 400*1e-3
        if pipdiff<1.1:
            pipdiff=1.1
            
        SLTPRatio = 2. #pipdiff*Ratio gives TP
        
        for i in range(1,barsupfront+1):
            value1 = close[line]-low[line+i]
            value2 = close[line]-high[line+i]
            valueOpenLow = max(value1, valueOpenLow)
            valueOpenHigh = min(value2, valueOpenHigh)

            if ( (valueOpenLow >= (SLTPRatio*pipdiff) ) and (-valueOpenHigh < pipdiff) ):
                trendcat[line] = 1 #-1 downtrend
                break
            elif ((valueOpenLow < pipdiff) ) and (-valueOpenHigh >= (SLTPRatio*pipdiff)):
                trendcat[line] = 2 # uptrend
                break 
            else:
                trendcat[line] = 0 # no clear trend
            
    return trendcat

# mytarget(barsfront to take into account, dataframe)
df['Trend'] = mytarget(100, df)
df.tail(20)

,Local time,open,high,low,close,volume,ATR,RSI,signal1,Trend
4277,23.09.2019 00:00:00.000 GMT+0300,107.736,107.771,107.311,107.555,154022.7693,0.576593,52.914147,0,0
4278,24.09.2019 00:00:00.000 GMT+0300,107.555,107.799,106.962,107.096,215279.3002,0.602633,47.253783,0,0
4279,25.09.2019 00:00:00.000 GMT+0300,107.099,107.886,107.002,107.780,246974.1095,0.630770,54.982115,0,0
4280,26.09.2019 00:00:00.000 GMT+0300,107.780,107.960,107.426,107.848,212912.0169,0.621093,55.677391,0,0
4281,27.09.2019 00:00:00.000 GMT+0300,107.849,108.182,107.659,107.995,222541.0272,0.611284,57.215721,0,0
4282,30.09.2019 00:00:00.000 GMT+0300,107.979,108.181,107.746,108.094,177440.6687,0.593655,58.266264,0,0
4283,01.10.2019 00:00:00.000 GMT+0300,108.095,108.472,107.629,107.757,215975.8300,0.618590,53.454611,0,0
4284,02.10.2019 00:00:00.000 GMT+0300,107.757,107.895,107.048,107.190,245226.7400,0.641431,46.497282,0,0
4285,03.10.2019 00:00:00.000 GMT+0300,107.190,107.300,106.486,106.928,254161.2999,0.658688,43.668934,0,0
4286,04.10.2019 00:00:00.000 GMT+0300,106.928,107.135,106.583,106.963,181875.1000,0.648019,44.157609,0,0


In [7]:
conditions = [(df['Trend'] == 1) & (df['signal1'] == 1),
              (df['Trend'] == 2) & (df['signal1'] == 2)]
values = [1, 2]
df['result'] = np.select(conditions, values)

trendId=2
print(df[df['result']==trendId].result.count()/df[df['signal1']==trendId].signal1.count())
df[ (df['result']!=trendId) & (df['signal1']==trendId) ] 

0.42696629213483145


,Local time,open,high,low,close,volume,ATR,RSI,signal1,Trend,result
234,26.03.2004 00:00:00.000 GMT+0200,106.160,106.232,105.506,106.012,6.129534e+05,0.986751,33.080490,2,1,0
295,21.06.2004 00:00:00.000 GMT+0300,108.785,108.977,108.228,108.941,6.138832e+05,1.321969,42.754302,2,0,0
378,14.10.2004 00:00:00.000 GMT+0300,109.773,109.849,109.147,109.691,6.010409e+05,0.823047,44.578319,2,1,0
439,07.01.2005 00:00:00.000 GMT+0200,104.992,105.125,103.858,104.753,5.940642e+05,1.218254,54.782867,2,1,0
1014,22.03.2007 23:00:00.000 GMT+0200,118.103,118.248,117.411,118.135,5.904808e+05,1.153799,49.872895,2,0,0
1093,12.07.2007 00:00:00.000 GMT+0300,122.490,122.600,121.840,122.430,4.221666e+06,0.830190,48.546249,2,1,0
1107,01.08.2007 00:00:00.000 GMT+0300,118.630,119.000,117.610,118.960,5.064306e+06,1.080985,32.322920,2,1,0
1111,07.08.2007 00:00:00.000 GMT+0300,118.900,118.990,117.980,118.850,3.556891e+06,1.169730,37.184327,2,1,0
1139,14.09.2007 00:00:00.000 GMT+0300,115.100,115.410,114.370,115.390,2.581671e+06,1.301338,46.721917,2,0,0
1175,05.11.2007 00:00:00.000 GMT+0200,114.760,114.820,114.025,114.565,3.399991e+06,0.980028,42.637528,2,1,0


In [11]:
dfpl = df[400:500]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.show()